<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/08_LS8_C2_T1_SR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract primary productivity indicies from Landsat 8, Collection 2, Tier 1, Surface Reflection (Level 2) data

## Import gee and authenticate

In [ ]:
!pip install geemap --quiet

In [ ]:
import geemap

In [ ]:
# This doesn't really work, it's only valid for the current runtime...

import os

# If credentials file doesn't exist, authenticate and store credentials
# Else if credentials file does exist, use stored credentials and initialise
if not os.path.exists(os.path.expanduser("~/.config/earthengine/credentials")):
  import ee
  ee.Authenticate()
  ee.Initialize()
else:
  import ee
  ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR")
aoi_name = "MKR"

# Layer (options: NDVI, EVI, MSAVI)
band_layer = "EVI"

# GEE layer ID
layer_dict = {
  "NDVI": "RS_017",
  "EVI": "RS_018",
  "MSAVI":"RS_019"
}

layer_name = layer_dict[band_layer]

# Image reducer (options: mean, median, min, max, stdDev, sum, product)
img_col_reducer = "mean"

# Date parameters
start_year = 2013
end_year = 2022

# Range doesn't include the stop value
year_list = ee.List(list(range(start_year, end_year+1)))

# Season parameters (months)
rain_start = 3
rain_end = 5
dry_start = 7
dry_end = 10

## Functions for Landsat processing

In [ ]:
# Scaling factors
def applyScaleFactors(image):
  opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  return image.addBands(opticalBands, None, True)

# Function to get and rename bands of interest from OLI.
def renameOLI(image):
    return image.select(
        ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL'],
        ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'QA_PIXEL'])

# Define function to mask out clouds and cloud shadows.
def fmask(image):
    cloudShadowBitMask = 1 << 3
    cloudsBitMask = 1 << 4
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask)

# Calculate and add NDVI band
def addNDVI(image):
    ndvi = image.normalizedDifference(['NIR', 'Red']).rename('NDVI')
    return image.addBands(ndvi)

# Calculate and add EVI
def addEVI(image):
  evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': image.select('NIR'),
        'RED': image.select('Red'),
        'BLUE': image.select('Blue')
    }).rename('EVI')
  return image.addBands(evi)

# Calculate and add MSAVI
def addMSAVI(image):
  msavi = image.expression(
    '(2 * NIR + 1 - sqrt(pow((2 * NIR + 1), 2) - 8 * (NIR - RED))) / 2', {
        'NIR': image.select('NIR'),
        'RED': image.select('Red')
    }).rename('MSAVI')
  return image.addBands(msavi)

# Prepare OLI mage (Landsat 8)
def prepOLI(image):
  orig = image
  image = applyScaleFactors(image)
  image = renameOLI(image)
  image = fmask(image)
  image = addNDVI(image)
  image = addEVI(image)
  image = addMSAVI(image)
  return ee.Image(image.copyProperties(orig, orig.propertyNames()))

#  Filter bounds needs to be first otherwise calculations take forever
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l8 = l8.filterBounds(aoi).map(prepOLI)

bandname = ee.String(l8.first().bandNames())
print(bandname.getInfo())
print(l8.first().getInfo())

## NDVI/MSAVI/EVI processing

In [ ]:
reducer_list = ee.Reducer.mean() \
.combine(reducer2 = ee.Reducer.median(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.min(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.max(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.stdDev(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.sum(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.product(), sharedInputs=True)


def annual_image(year_date):
  start = ee.Date.fromYMD(year_date, 1, 1)
  end = ee.Date.fromYMD(year_date, 12, 31)
  date_range = ee.DateRange(start, end)
  name = start.format('YYYY_MM').cat('_to_').cat(end.format('YYYY_MM'))
  return l8 \
        .filterDate(date_range) \
        .select(band_layer) \
        .reduce(reducer = reducer_list) \
        .clip(aoi) \
        .set({'name': name})

annual_image = year_list.map(annual_image)

## Check an element of list
year_mosaic = ee.Image(annual_image.get(1))
label = ee.String(year_mosaic.get('name')).getInfo()
print(label)
print(year_mosaic.getInfo())
print(year_mosaic.bandNames().getInfo())

## Check a reducer band
band_select = ".*" + img_col_reducer
print(band_select)
print(year_mosaic.select(band_select).getInfo())

In [ ]:
def annual_seasonal_image(year_date, season_start, season_end):
  start = ee.Date.fromYMD(year_date, season_start, 1)
  end = ee.Date.fromYMD(year_date, season_end, 30)
  date_range = ee.DateRange(start, end)
  season_label = "dry" if season_start == 7 else "wet"
  return l8 \
        .filterDate(date_range) \
        .select(band_layer) \
        .reduce(reducer = reducer_list) \
        .clip(aoi)  \
        .set({'season': season_label,
              'year': year_date})

def map_seasonal_dry(year):
  return annual_seasonal_image(year, dry_start, dry_end)

annual_dry = year_list.map(map_seasonal_dry)

def map_seasonal_rain(year):
  return annual_seasonal_image(year, rain_start, rain_end)

annual_rain = year_list.map(map_seasonal_rain)

## Map check

In [ ]:
ndvi_vis_params = {"min": 0.0, "max": 1.0,
                "palette": ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
                            '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
                            '012E01', '011D01', '011301']
                  }

Map = geemap.Map()
Map.addLayer(year_mosaic.select(band_select), ndvi_vis_params, 'NDVI')

# Map.addLayer(ndvi2011, ndvi_vis_params, 'NDVI-2011')
# Map.addLayer(ndvi2014, ndvi_vis_params, 'NDVI-2014')

Map.centerObject(aoi, zoom=10)
Map

## Export data - create task

In [ ]:
# Annual images
for i in  range(ee.List.length(annual_image).getInfo()):
  band_select = ".*" + img_col_reducer
  output_img =  ee.Image(annual_image.get(i))
  output_img = output_img.select(band_select)
  output_name = f"{layer_name}_{img_col_reducer}_{aoi_name}_{ee.String(output_img.get('name')).getInfo()}"

  task = ee.batch.Export.image.toDrive(image = output_img,
                                      region = aoi.geometry(),
                                      description = "EXPORT IMAGE TO DRIVE",
                                      folder = "GEE_exports",
                                      fileNamePrefix = output_name,
                                      scale = 30,
                                      maxPixels = 10e12,
                                      crs = "EPSG:4326"
                                      )
  task.start()
  print("STARTED TASK ", i+1)

In [ ]:
# Seasonal images - DRY
for i in range(ee.List.length(annual_dry).getInfo()):
  band_select = ".*" + img_col_reducer
  output_img =  ee.Image(annual_dry.get(i))
  output_img = output_img.select(band_select)
  output_name = f"{layer_name}_{img_col_reducer}_{aoi_name}_{ee.String(output_img.get('year')).getInfo()}_{ee.String(output_img.get('season')).getInfo()}"

  task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 30,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
  task.start()
  print("STARTED TASK ", "DRY ", i+1)

In [ ]:
# Seasonal images - RAIN
for i in range(ee.List.length(annual_rain).getInfo()):
  band_select = ".*" + img_col_reducer
  output_img =  ee.Image(annual_rain.get(i))
  output_img = output_img.select(band_select)
  output_name = f"{layer_name}_{img_col_reducer}_{aoi_name}_{ee.String(output_img.get('year')).getInfo()}_{ee.String(output_img.get('season')).getInfo()}"

  task = ee.batch.Export.image.toDrive(image = output_img,
                                     region = aoi.geometry(),
                                     description = "EXPORT IMAGE TO DRIVE",
                                     folder = "GEE_exports",
                                     fileNamePrefix = output_name,
                                     scale = 30,
                                     maxPixels = 10e12,
                                     crs = "EPSG:4326"
                                     )
  task.start()
  print("STARTED TASK ", "RAIN ", i+1)

## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:ee.List.length(year_list).getInfo()]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)